In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TFTrainingArguments, TFAutoModelForSequenceClassification, TFTrainer
import tensorflow as tf
import wandb

gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.9)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

wandb.login(key = '5d7ca03a263dd16d3b863633f7ddf7900842d17c')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gorden-ggm (gt). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Gorde/.netrc


True

In [2]:
df_depression_dataset = pd.read_csv('Dataset/dataset.csv')
df_depression_dataset.dropna(inplace = True)

df_train, df_test = train_test_split(df_depression_dataset, test_size = 0.2, random_state = 1)
df_train, df_valid = train_test_split(df_train, test_size = 0.1, random_state = 1)

X_train = df_train['clean_text'].values.tolist()
y_train = df_train['is_depression'].values.tolist()

X_valid = df_valid['clean_text'].values.tolist()
y_valid = df_valid['is_depression'].values.tolist()

X_test = df_test['clean_text'].values.tolist()
y_test = df_test['is_depression'].values.tolist()

In [3]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(X_train, truncation=True, padding=True)
valid_encodings = tokenizer(X_valid, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),y_train))
valid_dataset = tf.data.Dataset.from_tensor_slices((dict(valid_encodings),y_valid))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings),y_test))

In [4]:
training_args = TFTrainingArguments(
    output_dir = './results',         
    num_train_epochs = 3,              
    per_device_train_batch_size = 4,  
    per_device_eval_batch_size = 8,           
    weight_decay = 0.01,              
    logging_dir = './logs',           
    logging_steps = 10
)

with training_args.strategy.scope():
    model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

trainer = TFTrainer(
    model = model,                         
    args = training_args,                
    train_dataset = train_dataset,         
    eval_dataset = valid_dataset           
)    

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
trainer.train()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [9]:
prediction = trainer.predict(test_dataset)

In [12]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, prediction[1])
print(cm)

[[782   0]
 [  0 764]]


In [13]:
tf_save_directory = "Model\model_1"
tokenizer.save_pretrained(tf_save_directory)
model.save_pretrained(tf_save_directory)